In [1]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split
from sklearn import svm
import numpy as np
import os
import cv2
from joblib import dump, load

In [2]:
# Define the random seed
path_to_digits_dataset = r'digits_dataset2'
path_to_symbols_dataset = r'symbols_dataset'

random_seed = 42  # Ensures that you (or others) can run the code multiple times and get the same results.
target_img_size = (128, 128)
classifiers = {
    'SVM': svm.LinearSVC(random_state=random_seed),
}

def extract_hog_features(img):
    """
    Extracts Histogram of Oriented Gradients (HOG) features from the input image.
    This involves resizing the image to a fixed size, dividing it into cells and blocks, 
    computing gradient histograms, and flattening the result into a feature vector. 
    The extracted features are useful for machine learning tasks like image classification and detection.
    """
    img = cv2.resize(img, dsize=target_img_size)
    win_size = (128, 128) 
    cell_size = (16, 16) #Divides the window into smaller cells (4x4 pixels per cell).
    block_size_in_cells = (8, 8) 

    # divides window to blocks then blocks to cells
    block_size = (block_size_in_cells[1] * cell_size[1], block_size_in_cells[0] * cell_size[0])
    block_stride = (cell_size[1], cell_size[0]) #Determines how much the block moves at each step (4x4 pixels, matching cell size).
    nbins = 9  # Number of orientation bins
    hog = cv2.HOGDescriptor(win_size, block_size, block_stride, cell_size, nbins)
    h = hog.compute(img)
    h = h.flatten()
    return h.flatten() #This array is used as input for machine learning models.

def load_dataset(path_to_dataset):
    features = []
    labels = []
    img_filenames = os.listdir(path_to_dataset)

    for i, fn in enumerate(img_filenames):
        if fn.split('.')[-1] != 'jpg':
            continue

        label = fn.split('.')[0]
        labels.append(label)

        path = os.path.join(path_to_dataset, fn)
        img = cv2.imread(path)
        features.append(extract_hog_features(img))
        
        # show an update every 1,000 images
        if i > 0 and i % 1000 == 0:
            print("[INFO] processed {}/{}".format(i, len(img_filenames)))
        
    return features, labels   

def load_dataset_symbols(root_folder):
    features = []
    labels = []
    symbol_folders = os.listdir(root_folder)

    for symbol in symbol_folders:
        symbol_folder = os.path.join(root_folder , symbol)
        symbol_files = os.listdir(symbol_folder)
        # print(symbol_files)
        for filename in symbol_files:
            if filename.endswith(".jpg"):
                img_path = os.path.join(symbol_folder, filename)
                img = cv2.imread(img_path, 0)  # Load image in grayscale
                if img is not None:
                    features.append(extract_hog_features(img))
                    # print(features)
                    labels.append(symbol)
    return features, labels # should it be np.array(features) ??

# This function will test all our classifiers on the specifc SVM model and append the result to the SVM model 
def run_experiment(path_to_dataset, digits_or_symbols):
    
    # Load dataset with extracted features
    print('Loading dataset. This will take time ...')
    if digits_or_symbols == 0:
        features, labels = load_dataset(path_to_dataset)
    else:
        features, labels = load_dataset_symbols(path_to_dataset)
    print('Finished loading dataset.')
    
    # Since we don't want to know the performance of our classifier on images it has seen before
    # we are going to withhold some images that we will test the classifier on after training 
    train_features, test_features, train_labels, test_labels = train_test_split(
        features, labels, test_size=0.2, random_state=random_seed)
    digits_models = {}
    symbols_models = {}
    for model_name, model in classifiers.items():
        print('############## Training', model_name, "##############")
        # Train the model only on the training features
        model.fit(train_features, train_labels)
        
        # Test the model on images it hasn't seen before
        accuracy = model.score(test_features, test_labels)
        
        if digits_or_symbols == 0:
            digits_models[model_name] = model
        else:
            symbols_models[model_name] = model

        print(model_name, 'accuracy:', accuracy*100, '%')

        if digits_or_symbols == 0:
            dump(digits_models, 'digits_models.joblib')
            print('Saved digits models to digits_models.joblib')
        else:
            dump(symbols_models, 'symbols_models.joblib')
            print('Saved symbols models to symbols_models.joblib')
    if digits_or_symbols == 0:
        return digits_models
    
    return symbols_models

## You do not need to call these as the model file is already provided
run_experiment(path_to_digits_dataset, 0)
# run_experiment(path_to_symbols_dataset, 1)


Loading dataset. This will take time ...
[INFO] processed 1000/37949
[INFO] processed 2000/37949
[INFO] processed 3000/37949
[INFO] processed 4000/37949
[INFO] processed 5000/37949
[INFO] processed 6000/37949
[INFO] processed 7000/37949
[INFO] processed 8000/37949
[INFO] processed 9000/37949
[INFO] processed 10000/37949
[INFO] processed 11000/37949
[INFO] processed 12000/37949
[INFO] processed 13000/37949
[INFO] processed 14000/37949
[INFO] processed 15000/37949
[INFO] processed 16000/37949
[INFO] processed 17000/37949
[INFO] processed 18000/37949
[INFO] processed 19000/37949
[INFO] processed 20000/37949
[INFO] processed 21000/37949
[INFO] processed 22000/37949
[INFO] processed 23000/37949
[INFO] processed 24000/37949
[INFO] processed 25000/37949
[INFO] processed 26000/37949
[INFO] processed 27000/37949
[INFO] processed 28000/37949
[INFO] processed 29000/37949
[INFO] processed 30000/37949
[INFO] processed 31000/37949
[INFO] processed 32000/37949
[INFO] processed 33000/37949
[INFO] proc

{'SVM': LinearSVC(random_state=42)}